In [1]:
import utils
import mujoco
import os
import pickle
from scipy.io import savemat 
from dm_control import mjcf
import numpy as np
import jax
from jax import numpy as jnp
import time
from controller import *
import stac_base

# jax.disable_jit(disable=True)

/n/home05/charleszhang/.conda/envs/stac-mjx/lib/python3.12/site-packages/jaxfit/minpack.py:8: DeprecationWarning: Accessing jax.config via the jax.config submodule is deprecated.
  from jax.config import config
/n/home05/charleszhang/.conda/envs/stac-mjx/lib/python3.12/site-packages/jaxfit/least_squares.py:7: DeprecationWarning: Accessing jax.config via the jax.config submodule is deprecated.
  from jax.config import config
/n/home05/charleszhang/.conda/envs/stac-mjx/lib/python3.12/site-packages/jaxfit/trf.py:93: DeprecationWarning: Accessing jax.config via the jax.config submodule is deprecated.
  from jax.config import config
/n/home05/charleszhang/.conda/envs/stac-mjx/lib/python3.12/site-packages/jaxfit/common_jax.py:7: DeprecationWarning: Accessing jax.config via the jax.config submodule is deprecated.
  from jax.config import config
/n/home05/charleszhang/.conda/envs/stac-mjx/lib/python3.12/site-packages/jaxfit/loss_functions.py:1: DeprecationWarning: Accessing jax.config via the 

In [2]:

# If your machine is low on ram:
# os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION'] = '.6'
# os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = "false"

def get_clip(kp_data, n_frames):
    import random
    max_index = kp_data.shape[0] - n_frames + 1
    rand_start = random.randint(0, max_index)
    return kp_data[rand_start:rand_start+n_frames,:]



In [3]:
utils.init_params("../params/params.yaml")

In [4]:
rat_xml = "../models/rodent_stac.xml"
rat23 = "../models/rat23.mat"
# data_path = "save_data_AVG.mat"
data_path = "/n/holylabs/LABS/olveczky_lab/holylfs02/Everyone/dannce_rig/dannce_ephys/art/2020_12_22_1/DANNCE/predict03/save_data_AVG.mat" 

fit_path = "floating_fit.p"
transform_path = "floating_transform.p"
utils.params['FTOL'] = 1e-05
utils.params['n_fit_frames'] = 500
utils.params['N_ITERS'] = 1
skip_transform = True

In [5]:
model = mujoco.MjModel.from_xml_path(rat_xml)
model.opt.solver = mujoco.mjtSolver.mjSOL_NEWTON
model.opt.disableflags = mujoco.mjtDisableBit.mjDSBL_EULERDAMP
model.opt.iterations = 1
model.opt.ls_iterations = 4

In [ ]:
start_time = time.time()

root = mjcf.from_path(rat_xml)

# Default ordering of mj sites is alphabetical, so we reorder to match
kp_names = utils.loadmat(rat23)["joint_names"]
utils.params["kp_names"] = kp_names

# argsort returns the indices that would sort the array
stac_keypoint_order = np.argsort(kp_names)
# Load kp_data, /1000 to scale data (from mm to meters i think?)
kp_data = utils.loadmat(data_path)["pred"][:] / 1000

kp_data = prep_kp_data(kp_data, stac_keypoint_order)

# setup for fit
physics, mj_model = set_body_sites(root)
part_opt_setup(physics)

# Run root optimization
# Create mjx model and data
mjx_model = mjx.put_model(mj_model)
mjx_data = mjx.make_data(mjx_model)

# Get and set the offsets of the markers
offsets = jnp.copy(stac_base.get_site_pos(mjx_model))
offsets *= utils.params['SCALE_FACTOR']

# print(mjx_model.site_pos, mjx_model.site_pos.shape)
mjx_model = stac_base.set_site_pos(mjx_model, offsets)

# forward is used to calculate xpos and such
mjx_data = mjx.kinematics(mjx_model, mjx_data)
mjx_data = mjx.com_pos(mjx_model, mjx_data)

print("start")

mjx_data = root_optimization(mjx_model, mjx_data, kp_data)

start
Root Optimization:
data.qpos: [2.78957345e-02 1.85267244e-07 6.12806154e-02 1.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.0

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x145b35451970>>
Traceback (most recent call last):
  File "/n/home05/charleszhang/.conda/envs/stac-mjx/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


In [ ]:
scene_option = mujoco.MjvOption()
scene_option.geomgroup[2] = 1
scene_option.sitegroup[2] = 1
scene_option.sitegroup[3] = 1
scene_option.flags[enums.mjtVisFlag.mjVIS_TRANSPARENT] = True
scene_option.flags[enums.mjtVisFlag.mjVIS_LIGHT] = False
scene_option.flags[enums.mjtVisFlag.mjVIS_CONVEXHULL] = True
# render the frame (frame 0)
renderer = mujoco.Renderer(model)
mj_data = mjx.get_data(mj_model, mjx_data)
renderer.update_scene(mj_data, scene_option=scene_option)
pixels = renderer.render()